In [1]:
import os
import pandas as pd
import numpy as np
import datetime
from dateutil import parser
import dateutil
from sklearn.cross_validation import train_test_split
from utils import *
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import pylab
from compALS import *
from collections import Counter
from sklearn.neighbors import NearestNeighbors


print('Importing data.')
ratingData,movieData,userData = importData("/home/satvik/Analytics/Recommender Project/")


Importing data.


In [2]:
neighborCount=5

In [3]:
data = pd.merge(ratingData,userData,how="left",on="userID")
data = pd.merge(data,movieData,how="left",on="movieID")
data = cleanData(data)  #removes unneccesary rows and columns
data = dateTime(data)

#%%

data_sub = data[['userID','movieID','rating']]

pivot = data_sub.pivot_table('rating','userID','movieID')
pivot=pivot.fillna(0)
#%%

mask = (pivot.values>0.5).astype("int")

sparseData = csr_matrix(pivot.values)
als = nf.Lsnmf(sparseData,seed="random_vcol",rank=2,max_iter=30,beta=0.5)
als_fit= als.factorize()

user_features = np.array(als_fit.basis().todense())
movie_features = np.array(als_fit.coef().todense())

dataset = pd.DataFrame(user_features)


In [4]:
neighborModel = NearestNeighbors()
nearestNeighbors = neighborModel.fit(X=movieData.ix[:,6:-1])
nearest= nearestNeighbors.kneighbors(movieData.ix[:,6:-1],n_neighbors=neighborCount,return_distance=False)[:,0:neighborCount]

In [12]:
userID = 34
movierecs = np.zeros(len(nearest))
arr = pivot.values[userID,:]
for i in range(len(arr)):
    if arr[i]>0:
        for j in range(len(nearest[0])):
            movierecs[nearest[j]] = movierecs[nearest[j]] + arr[i]
print movierecs.argsort()

[ 840 1125 1124 1123 1122 1121 1120 1119 1118 1117 1116 1126 1115 1113 1112
 1111 1110 1109 1108 1107 1106 1105 1104 1114 1103 1127 1129 1151 1150 1149
 1148 1147 1146 1145 1144 1143 1142 1128 1141 1139 1138 1137 1136 1135 1134
 1133 1132 1131 1130 1140 1152 1102 1100 1073 1072 1071 1070 1069 1068 1067
 1066 1065 1064 1074 1063 1061 1060 1059 1058 1057 1056 1055 1054 1053 1052
 1062 1101 1075 1077 1099 1098 1097 1096 1095 1094 1093 1092 1091 1090 1076
 1089 1087 1086 1085 1084 1083 1082 1081 1080 1079 1078 1088 1153 1154 1155
 1231 1229 1228 1227 1226 1225 1224 1223 1222 1221 1232 1220 1218 1217 1216
 1215 1214 1213 1212 1211 1210 1209 1219 1208 1233 1235 1257 1256 1255 1254
 1253 1252 1251 1250 1249 1248 1234 1247 1245 1244 1243 1242 1241 1240 1239
 1238 1237 1236 1246 1207 1206 1205 1177 1176 1175 1174 1173 1172 1171 1170
 1169 1168 1178 1167 1165 1164 1163 1162 1161 1160 1159 1158 1157 1156 1166
 1179 1180 1181 1204 1203 1202 1201 1200 1199 1198 1197 1196 1195 1194 1193
 1192 1191 1